MAIN

Each part of the following script is used to proccess the Raw data stept by step. 
Read and follow the instructions carefully.

1. Zone Creation

    Run this cell to introduce the name of the folder where all the data is goint to be downloaded.
       To avvoid future errors it would rather to introduce names without spaces.
       For European country names use the ISO 3166-1 standars i.e. AT, BE, BG, CH.... etc.
       For the folder path is recommendable keep the Dispa-SET directory, e.g.
           /dispa-SET/home/directory/path/RawData/PowerPlants
           home/ray/Dispa-SET_Unleash/RawData/PowerPlants

In [14]:
# Input variables from user
zone_folder_name = input("Enter the zone folder name: ")
power_plants_folder_path = input("Enter the power plants folder path: ")

# Call the create_folder.py script using %run
%run Zone_Folder_Creation.py {zone_folder_name} {power_plants_folder_path}

Enter the zone folder name:  TEST
Enter the power plants folder path:  /home/ray/Dispa-SET_Unleash/RawData/PowerPlants


Folder 'TEST' created at: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/TEST
Zone folder path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/TEST


2. Power Plants Data download

    Run this cell to download files containers of raw data related power plants units of the zone previously created.
    To keep the Dispa-SET directories structure and name of files, is recommended to give the year of the data collected as the downloaded file name. Additionally there may be different power plant data sources for an only country, zone, region, etc, is also recommended to add a numeric suffix to the file name to difference between files e.g.
        user_filename = "20##-##"
        user_filename = "2020-01"

In [15]:
# Retrieve zone_folder_path from the output of the first cell (assuming it's stored in a variable named 'zone_folder_path')
# If zone_folder_path is stored in a file, you would read it from the file here instead
# For example:
# with open("zone_folder_path.txt", "r") as file:
#     zone_folder_path = file.read()

# Now you can use zone_folder_path as needed in this cell and subsequent cells
print("Zone folder path:", zone_folder_path)

# Input file name and download link from user
power_plants_file_name = input("Enter the file name: ")
download_link = input("Enter the download link: ")

# Call the download_script.py script using %run
%run Power_Plants_Data_download.py {power_plants_file_name} {download_link} {zone_folder_name} {zone_folder_path}

Zone folder path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/TEST


Enter the file name:  2020-01
Enter the download link:  https://opendata.elia.be/api/explore/v2.1/catalog/datasets/ods036/exports/csv?lang=en&timezone=Europe%2FBrussels&use_labels=true&delimiter=%3B


CSV file '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/TEST/TEST_sources.csv' created with the download link
File '2020-01' downloaded and saved at: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/TEST/2020-01
